### Part 3: Create the Search Agent

Now that we've vectorized our health plan documents and created an index, we can build a Search Agent with Microsoft Agent Framework. This agent will leverage the Azure AI Search index to retrieve relevant information from your health plan documents in response to user queries. In this section, you'll:
- Connect your code to the Azure AI Search index you created earlier
- Define and configure a search agent with the appropriate tools and resources
- Interact with the agent to perform intelligent, context-aware document retrieval

This hands-on exercise demonstrates how retrieval-augmented generation (RAG) can be implemented programmatically, enabling your agent to provide accurate, document-grounded answers using Azure AI services.

In [ ]:
import os
import json
from dotenv import load_dotenv
# uv add the following:
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from azure.core.exceptions import HttpResponseError

from agent_framework.azure import AzureOpenAIChatClient

load_dotenv()

# Your existing configuration
deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")
search_key = os.getenv("SEARCH_API_KEY")
index_name = os.getenv("INDEX_NAME")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")

For the following exercise, make sure to add the following environment variables to your .env file:

```python
AZURE_SEARCH_ENDPOINT=https://<search-service-name>.search.windows.net
INDEX_NAME=<index-name>-index
SEARCH_API_KEY=<search-service-key>
```

You’re creating two clients:

- AzureOpenAIChatClient connects to your Azure OpenAI deployment using the endpoint, API key, version, and deployment name so you can send prompts to the model.
- SearchClient connects to your Azure AI Search index using its endpoint, index name, and key, allowing you to retrieve relevant documents.

Together, these enable a RAG workflow: fetch context from search, then pass it to the model for grounded answers.

In [ ]:
# Create the client using API key
client = AzureOpenAIChatClient(
    endpoint=endpoint,
    api_key=api_key,
    api_version=api_version,
    deployment_name=deployment
)

# Initialize search client with correct credentials
search_client = SearchClient(
    endpoint=search_endpoint,
    index_name=index_name,
    credential=AzureKeyCredential(search_key)
)

The `AzureAISearchTool` wrapper that turns your SearchClient into a callable “tool” for an agent. It’s initialized with an existing Azure AI Search SearchClient. 

The search(query, top) method runs a semantic search against your index, collects each hit’s chunk content and @search.score, and returns them as a JSON-formatted string for easy injection into a prompt. If Azure Search returns an error (HttpResponseError), it prints basic diagnostics and returns a JSON error payload instead:

In [ ]:
class AzureAISearchTool:
    """Function tool to search Azure AI Search index"""
   
    def __init__(self, search_client: SearchClient):
        self.search_client = search_client
   
    def search(self, query: str, top: int = 5) -> str:
        """
        Search the Azure AI Search index for relevant documents
       
        Args:
            query: The search query about lighting technology advancements or related topics
            top: Number of results to return (default 5)
       
        Returns:
            JSON string containing the search results with chunks and metadata
        """
        try:
            results = self.search_client.search(
                query,
                top=top,
                query_type="semantic"
            )
           
            documents = []
            for doc in results:
                documents.append({
                    "chunk": doc.get('chunk', ''),
                    "score": doc.get('@search.score', 0)
                })
           
            # Return formatted results
            return json.dumps(documents, indent=2)
           
        except HttpResponseError as e:
            print("HTTP status:", e.status_code if hasattr(e, "status_code") else "N/A")
            if e.response is not None:
                try:
                    print("Response text:", e.response.text())
                except Exception:
                    print("Could not get response.text()")
            return json.dumps({"error": str(e)})

Finally, we create a simple agent and run it with a user question. The agent uses your Azure OpenAI model for reasoning and the Azure AI Search tool for retrieval. It follows the given instructions to stay factual and cite sources. 

When you run the cell, you should see a grounded response based on the indexed documents:

In [ ]:
# Initialize the search tool for the agent providing the search client
tools = AzureAISearchTool(search_client=search_client)

agent = client.create_agent(
    instructions="""
    Assistant helps with questions on light technology and lighting systems. Be brief in your answers.
    Answer ONLY with the facts listed in the sources you retrieve.
    If there isn't enough information, say you don't know. Do not generate answers that don't use the retrieved sources.
    Include source references for each fact you use using square brackets.""",
    
    tools=[tools.search]
)

# Minimal run sample with a single question
user_question = "Can you tell me something about the impact of indoor lighting on human health?"
result = await agent.run(
    user_question,
    )

print(result)